**_This notebook concentrates on basic data cleaning required to use the data for all the future activities._**

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('/Workspace/Users/deepak.guru.ganesan@volvo.com/leovegas/dataset.csv')
df.sort_values(['bet_placement_hour'], inplace=True)

#### Data Cleaning

In [0]:
#Wherever the country is replaced with league, they are interchanged¨
df['temp_league'] = np.where(df['league'].isna(), df['event_country'], df['league']) 
df['event_country'] = np.where(df['league'].isna(), 'Unknown', df['event_country'])
df = df.drop('league', axis=1)
df = df.rename(columns={'temp_league':'league'})

#Tried to create country for the most famous league
df['event_country'] = np.where(df['league'].str.contains('NBA'), 'USA', df['event_country'])

#Removed Milliseconds in teh Event Start date there by solving minor descrepancies with different start time for strange events
df['eventStartDate'] = pd.to_datetime(df['eventStartDate'], format='%Y-%m-%d %H:%M:%S').dt.floor('S')
df['bet_placement_hour'] = pd.to_datetime(df['bet_placement_hour'])

#Different names for same event correction
df['eventName'] = df.groupby('eventId')['eventName'].transform('first')

df.head()

,bet_placement_hour,eventId,eventName,eventStartDate,event_country,number_of_bets,total_turn_over_EUR,league
0,2023-12-01,1020157185,Oklahoma Sooners - Arkansas-Pine Bluff Golden ...,2023-12-01 01:00:00,Unknown,3,49.582521,NCAAB
36,2023-12-01,1020030711,New York Knicks - Detroit Pistons,2023-12-01 00:41:20,USA,249,8841.198662,NBA
37,2023-12-01,1020030716,Dallas Mavericks - Memphis Grizzlies,2023-12-02 00:40:47,USA,8,0.296827,NBA
38,2023-12-01,1019711357,Cairns Taipans - Brisbane Bullets,2023-12-01 08:30:00,Australia,4,22.363404,NBL
39,2023-12-01,1020030718,Oklahoma City Thunder - Los Angeles Lakers,2023-12-01 01:10:56,USA,14,8.513354,NBA


##### Event Start date cleaning

In [0]:
## Seconds Cleaning 
df = df.groupby(['eventId', 'bet_placement_hour','eventName','league','event_country','eventStartDate']).agg({'number_of_bets': 'sum','total_turn_over_EUR': 'sum'}).reset_index()

##EvenStart date hours cleaning
df['eventStartDate'] = df.groupby(['eventId'])["eventStartDate"].transform("max")
df = df.groupby(['eventId', 'bet_placement_hour','eventName','league','event_country','eventStartDate']).agg({'number_of_bets': 'sum','total_turn_over_EUR': 'sum'}).reset_index()

df = df[(df["number_of_bets"] != 0)]  ###Removing rows where the number of bets is 0
print(len(df))

228954


In [0]:
df.to_csv('/Workspace/Users/deepak.guru.ganesan@volvo.com/leovegas/dataset_cleaned.csv')